### PARTNERS: TUCKER SIMPSON, MARJORIE MACDONALD

# Problem 1. (45 points)
In this exercise, you are supposed to code DFS and related algorithms done in class. To help you, we are
providing Python code for a graph class file graph_class.py which you can import. You should feel
completely free to modify this code as you please. The operations that this gives you are:

* init: G = graph() would create an empty graph
* Read Edges: G.Read Edges(fname) would read directed edges from the file “fname” where eachline would contains endpoints of one directed edge separated by whitespace.
* Other routines are sufficiently commented to be clear. Ask questions on Ed Discussion for clarification

In [137]:
# import in graph class
from graph_class import graph

## a. Implement DFS. 
Read the directed graph in the file graph2.txt and run DFS on it. When you implement DFS, you should traverse neighbors (the For-loop of DFS) in alphabetical order1. The order σ in which you run DFS should also be alphabetical. So in particular, you should start exploring from the letter a. Write down the order in which the vertices are visited. Do you agree? (10 points)

In [2]:
'''read in graph from graph2.txt'''
# initalize graph
G = graph()
# read in edges
G.Read_Edges("graph2.txt")
G.Read_Vertex_Labels("graph2.txt")

In [3]:
# check to make sure has read in correctly
print(G.num_vertices(),G.num_edges())

13 20


In [354]:
from collections import deque

In [355]:
def DFS(G, ordering):
    global vert_to_index; vert_to_index = {}
    idx = 0
    for vert in ordering:
        vert_to_index[vert] = idx
        idx += 1
    
    global visited; visited = set()
    global F; F = graph()
    global first; first = [0] * G.num_vertices()
    global last; last = [0] * G.num_vertices()
    global root; root = [0] * G.num_vertices()
    global Fcomp; Fcomp = [0] * G.num_vertices()
    global time; time = 0
    global fcomp; fcomp = 0
    global parent; parent = {}
    
    for vert in ordering:
        if vert not in visited:
            root[fcomp] = vert
            fcomp += 1
            DFS_on_vert(G, vert)
    
    return (last, fcomp, Fcomp, first, vert_to_index, parent)
            
def DFS_on_vert(G, v): 
    global vert_to_index; global visited; global F; global first; global last
    global root; global Fcomp; global time; global fcomp; global parent

    stack = deque()
    
    # In format (vertex, is ghost copy?)
    stack.append((v, False))
    
    parent[v] = None
        
    while len(stack) > 0:
        # Now we need to remove it from the parent stack
        
        curr = stack.pop()
        x = curr[0]
        
        if x not in visited:
            visited.add(x)
            time += 1
            first[vert_to_index[x]] = time
            F.Add_Edge(parent[x], x)
            Fcomp[vert_to_index[x]] = fcomp
            stack.append((x, True))
            
            if x in G.adj_list and len(G.adj_list[x]) > 0:
                curr_adj_list = sorted(G.adj_list[x], reverse=True)

                for u in curr_adj_list:
                    if u not in visited:  
                        parent[u] = x

                        stack.append((u, False))
                    
        else: 
            if curr[1] == True:
                time += 1
                last[vert_to_index[x]] = time
            else:
                continue

In [356]:
'''read in graph from graph2.txt'''
# initalize graph
G = graph()
# read in edges
G.Read_Edges("graph2.txt")
G.Read_Vertex_Labels("graph2.txt")

order = sorted(G.vertices)
print(order)
DFS(G, order)

global first
print(first)

global last
print(last)

['a', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'r', 's', 't', 'u']
[1, 21, 3, 11, 9, 2, 13, 23, 4, 6, 14, 10, 22]
[20, 26, 8, 12, 18, 19, 16, 24, 5, 7, 15, 17, 25]


We ran this by hand, and we agree with our results.

## b. Using DFS routine, implement the O(n + m) time algorithm which detects whether a graph has a cycle or not, and if so returns the cycle. 
Read the directed graph in the file graph3.txt and run your code on it. Write down what you get. (10 points)

In [135]:
'''
Input: Graph G
Output: True  and the cycle if G contains one, false otherwise
'''
def check_for_cycle(G):
    order = G.vertices
    res = DFS(G, order)
    last = res[0]
    first = res[3]
    vert_to_index = res[4]
    
    for u in G.adj_list:
        for v in G.adj_list[u]:
            # We have edge (u,v)
            # If first[v] < first[u] <last[u] < first[v]
            if first[vert_to_index[v]] < first[vert_to_index[u]] and last[vert_to_index[u]] < last[vert_to_index[v]]:
                # Recover cycle
                # get order of vertices with v as the source
                # recover path from v to u and add (u,v)
                order = list(G.vertices)
                for i in range(len(order)):
                    if order[i] == v:
                        temp = order[0]
                        order[0] = order[i]
                        order[i] = temp
                        break;
                
                res2 = DFS(G, order)
                parent = res[5]
                vert_to_index = res2[4]
                
                curr = u
                
                cycle = []
                
                while curr != v:
                    cycle = [curr] + cycle
                    # print("Taking edge: (" + str(parent[curr]) + ", "+ str(curr) + ")")
                    curr = parent[curr]
                    # print(cycle)
                    
                cycle = [v] + cycle
                cycle = [u] + cycle
                
                
                return True, cycle
    
    return False, None

In [136]:
# initalize graph
G = graph()
# read in edges
G.Read_Edges("graph3.txt")
G.Read_Vertex_Labels("graph3.txt")

print(check_for_cycle(G))

(True, ['o', 'n', 'g', 'r', 'a', 't', 's', 'c', 'o'])


So, **there is a cycle**, and it is the walk {o, {o,n}, n, {n,g}, g, {g, r}, r, {r,a}, a, {a,t}, t, {t,s}, s, {s, c}, c, {c, o}, o}.

## c. Implement the algorithm to find strongly connected components in a directed graph. 
For each of the following graphs, return the (a) number of strongly connected components, and (b) the number of vertices in the SCC in the one with the maximum number of vertices, and (c) how many seconds it took to run your code. Run it on a toy example to see if it’s running correctly. For instance, on graph2.txt, the number of SCCs are 7 and the largest one 5 vertices. You may not be able to run on the second graph if you use the recursive implementation of DFS. The 10 points are for the stack implementation.

 * epinions.txt (has 130K nodes and 840K edges) (5 points)
 * googlemaps.txt (has 2M nodes and 2.8M edges) (10 points)

In [57]:
'''Reverses all the edges of the graph as subroutine '''

def ReverseEdges(G):
    # initalize empty graph to be gprime
    Gprime = graph()
    for v in G.vertices:
        for u in G.adj_list[v]:
            Gprime.Add_Edge(u, v)
    return Gprime

In [58]:
''' Subroutine to get the decreasing order of last[v]'s '''
def DecOrder(last, order):
    enumerated = sorted(enumerate(last), key=lambda k: k[1], reverse=True)
    
    indexes = []
    pi = []
    for i in enumerated:
        indexes.append(i[0])
    for j in indexes:
        pi.append(order[j])
    return pi

In [72]:
import time as real_time

In [77]:
'''Returns strongly connected components of G,
in topological order of G^SCC'''
def StronConnComp(G):
    start_time = real_time.time()
    # run DFS on G to get the last for every vertex
    last = DFS(G, list(G.vertices))[0] ### CHANGE SO THAT ACCESSES LAST VALUE
    # get the decreasing order of last[v]'s
    pi = DecOrder(last, list(G.vertices))
    # obtain G_rev
    GRev = ReverseEdges(G)
    # run DFS on GRev in the order of what is connected
    res = DFS(GRev, pi)
    SCC = res[1]
    FComp = res[2]
    print("We have " + str(SCC) + " connected components")
    
    start = 0
    end = 0
    biggest = 1
    
    while end < len(FComp):
        while end < len(FComp) and FComp[start] == FComp[end]:
            end += 1
            # print("Curr window: " + str(FComp[start:end]))
        biggest = max(biggest, end-start)
        start = end
    
    print("We have " + str(biggest) + " elements in our largest SCC.")
    end_time = real_time.time()
    tot_time = end_time - start_time
    print("It took " + str(tot_time) + " seconds to get these results.")

### (c.i) Strongly Conected on Epinions: 

In [138]:
'''read in graph from Epinions.txt'''
# initalize graph
print("Epinions test")
G = graph()
# read in edges
G.Read_Edges("epinions.txt")
G.Read_Vertex_Labels("epinions.txt")
StronConnComp(G)

Epinions test
Error: Vertex not in Graph
We have 88361 connected components
We have 31 elements in our largest SCC.
It took 11.024072885513306 seconds to get these results.


### (c.ii) Strongly Connected on Google Maps:

In [81]:
'''read in graph from googlemaps.txt'''
# initalize graph
print("Google Maps test")
G = graph()
# read in edges
G.Read_Edges("googlemaps.txt")
G.Read_Vertex_Labels("googlemaps.txt")
StronConnComp(G)

Google Maps test
We have 2638 connected components
We have 1957027 elements in our largest SCC.
It took 56.1551148891449 seconds to get these results.


## d. Implement a subroutine which takes a directed graph $G$ (not necessarily acyclic) as input and finds as long a directed path as it can. 
Also implement a subroutine check_path(G,p) which takes a graph G and a list p and returns TRUE if p is indeed a path in G: so, all entries of p are vertices, nothing is repeated in $p$, and adjacent vertices form edges. This will let you check if the path you return is indeed a path in the graphs or not. In the iPython/pdf file you must first clearly and succinctly describe the algorithm you will be using to implement the above longest path routine.

### Succinct Algorithm Description: 
We randomize the order in which we run DFS in order to find a topological order. Then, using this topological order, we remove all backedges from the graph. Then, using this newly formed DAG, we run longest path. We call this multiple times, and maximize over the value returned, taking advantage of randomness. All this is wrapped in a method that also checks if the path exists and captures time. 

In [346]:
''' 
takes graph G and list p and returns TRUE if p is a path in G
input: graph G, array p[1:n] where n is number of verticies in the path, and there is an edge
from all verticies {n-1, n}
'''
def check_path(G, p):
    # initalize counter to store location in array
    counter = 0
    # initalize empty set of vertices that have been "seen"
    A = set()
    # go through all vertices in the path
    for i in p:
        # make sure nothing repeated; if vertex in A, has already been seen and is duplicate
        if i in A:
            return False
        # check to see that it is a vertex in graph
        if not G.isVertex(i):
            return False
        # check that the graph contains a vertex to the right
        # unless it is the last one
        # if we are not looking at the last item in our path vertex array
        if not counter == len(p)-1:
            # check if there is an edge from our vertex to the vertex to the right of it
            # if there is not, return false
            if not G.isEdge(i, p[counter+1]):
                return False
        
        A.add(i)
        counter = counter + 1
    return True

In [340]:
'''
Input: Graph G and an order to run DFS
Output: Adjacency list of the graph that maps vertices to in-neighbors
    and topological order pi
'''
def graph_no_back_edge_flipped(G, order):
    #if edge pointing to vertex already seen then delete it 
    last = DFS(G, order)[0]

    # get the decreasing order of last[v]'s
    pi = DecOrder(last, order)

    # initalize new graph to store all edges that are not back edges 
    # REMOVE ALL BACK EDGES FROM GRAPH
    
    seen = set()

    new_graph = graph()

    for v in pi:
        seen.add(v)
        # if not already, add vertex to graph
        for u in G.adj_list[v]:
            # check if it's already a vertex in the graph
            # if it is, don't add it, or the edge, since that means it's a back edge
            # if it is not yet in the graph, it is a forward edge! 
            if u not in seen:
                new_graph.Add_Vertex(u)
                new_graph.Add_Edge(u, v)

    return new_graph.adj_list, pi  

In [187]:
import random

In [439]:
'''
Input: directed graph G and the longest path we've seen so far
Output: longest path seen and its length
'''
def longest_path(G, seen = set()):
    # Permute G and remove back edges
    order = list(G.vertices)
    random.shuffle(order)
    print("Order 0 = " + str(order[0]))
#     if order[0] in seeen:
#         return 0, [], None, []
    
    res = graph_no_back_edge_flipped(G, order)
    
    adj_map = res[0]
    pi = res[1]
    print("Pi 0: " + str(pi[0]))
    
    # Translator from letter world to index world
    vert_to_index = {}
    idx = 0
    for vert in pi:
        vert_to_index[vert] = idx
        idx += 1
        
    s = pi[0]
    
    # base case -- all paths are at least 1 vertex
    longest = [1] * G.num_vertices()
    parent = {}
    
    # keep track of the longest path we've seen and keep a pointer to the best vertex we've seen
    max_length = 0
    max_end = None
    
    # Go over all vertices in the topological order
    for j in range(len(pi)):
        # Go over all in-neighbors
        if pi[j] in adj_map:
            for l in adj_map[pi[j]]:
                l = vert_to_index[l]
                # Found a new longest path -- remember its length and where it came from
                if longest[l] + 1 > longest[j]:
                    longest[j] = longest[l] + 1
                    parent[j] = l
                    
        if j not in parent:
            parent[j] = None
        
        # Update longest path seen so far
        if max_length < longest[j]:
            max_length = longest[j]
            max_end = j        
    
    return max_length, parent, max_end, pi

In [429]:
'''
Input: End vertex of longest path, parents list, and the topological
    from the best search
Output: the longest path
'''
def recover_longest_path(end, parents, pi):
    path = deque()
    curr = end
    
    in_path = set()
    while curr != None:
        if curr in in_path:
            break
        in_path.add(curr)
        path.appendleft(pi[curr])
        curr = parents[curr]
    
    return list(path)

In [433]:
'''
Input: Directed graph G
Output: total time searched, longest path length, longest path found
Returns the longest valid path found in G
'''
def iterative_longest_path(G, num_calls):
    # Keep track of time
    start_time = real_time.time()
    
    # keep track of max length and best path
    max_len = 0
    best_parents = None
    best_end = None
    pi = None
    
    # Conduct num_calls searches of longest path
    for i in range(num_calls):
        res = longest_path(G)
        if res[0] > max_len:
            print("New longest path: " + str(res[0]))
            max_len = res[0]
            best_parents = res[1]
            best_end = res[2]
            pi = res[3]
    
    best_path = recover_longest_path(best_end, best_parents, pi)
    
    # Make sure we are returning a valid path of the correct length
    if check_path(G, best_path):
        if len(best_path) == max_len:
            tot_time = real_time.time() - start_time
            print("Longest Path: " + str(max_len))
            print("Total time searched: " + str(tot_time) + " seconds")
            return tot_time, max_len, best_path
        else:
            return "Error - path is incorrect length"
    
    else:
        return "Error - path is invalid"

### Next, read the directed graphs in the files challenge1.txt and challenge2.txt 
and for each of them write (a) what is the length of the path returned by your algorithm, (b) did check_path certify this path, and (c) how long (in seconds) did your code take to do so. Furthermore, in your code you also have to write what your paths are as a comment (so that the graders can check by running check_path).

### Challenge 1:

In [435]:
G = graph()
# read in edges
G.Read_Edges("challenge1.txt")
G.Read_Vertex_Labels("challenge1.txt")

iterative_longest_path(G, 500)

New longest path: 3365
New longest path: 3394
['3352', '1684', '107', '0', '101', '180', '194', '1', '119', '100', '163', '173', '166', '198', '1902', '1006', '1039', '1040', '1063', '1056', '1029', '1059', '1004', '1003', '1075', '1012', '1113', '1025', '1080', '1001', '1095', '1300', '1179', '1114', '1155', '1026', '1028', '1017', '1024', '1078', '1074', '1048', '1110', '1047', '1076', '1079', '1086', '1032', '1014', '1009', '1066', '1018', '1099', '1081', '1020', '1102', '1108', '1235', '1015', '1021', '1007', '1067', '1019', '1060', '1100', '1036', '1129', '1082', '2596', '1104', '1044', '1158', '1215', '1192', '1120', '1115', '1190', '1148', '1227', '1159', '1010', '1000', '1134', '1127', '1090', '1122', '1237', '1030', '1043', '1045', '1183', '1051', '1042', '1052', '1011', '1041', '1072', '1037', '1035', '1013', '1109', '1070', '1092', '1054', '1128', '1049', '1112', '1184', '1083', '1091', '1207', '1068', '1153', '1107', '1101', '1069', '1238', '1117', '1124', '1126', '1132', '

(143.0460340976715,
 3394,
 ['3352',
  '1684',
  '107',
  '0',
  '101',
  '180',
  '194',
  '1',
  '119',
  '100',
  '163',
  '173',
  '166',
  '198',
  '1902',
  '1006',
  '1039',
  '1040',
  '1063',
  '1056',
  '1029',
  '1059',
  '1004',
  '1003',
  '1075',
  '1012',
  '1113',
  '1025',
  '1080',
  '1001',
  '1095',
  '1300',
  '1179',
  '1114',
  '1155',
  '1026',
  '1028',
  '1017',
  '1024',
  '1078',
  '1074',
  '1048',
  '1110',
  '1047',
  '1076',
  '1079',
  '1086',
  '1032',
  '1014',
  '1009',
  '1066',
  '1018',
  '1099',
  '1081',
  '1020',
  '1102',
  '1108',
  '1235',
  '1015',
  '1021',
  '1007',
  '1067',
  '1019',
  '1060',
  '1100',
  '1036',
  '1129',
  '1082',
  '1104',
  '1044',
  '1158',
  '1215',
  '1192',
  '1120',
  '1115',
  '1190',
  '1148',
  '1227',
  '1159',
  '1010',
  '1000',
  '1134',
  '1127',
  '1090',
  '1122',
  '1237',
  '1030',
  '1043',
  '1045',
  '1183',
  '1051',
  '1042',
  '1052',
  '1011',
  '1041',
  '1072',
  '1037',
  '1035',
  '1013',

In [438]:
G = graph()
# read in edges
G.Read_Edges("challenge1.txt")
G.Read_Vertex_Labels("challenge1.txt")

iterative_longest_path(G, 1000)

New longest path: 3344
New longest path: 3360
New longest path: 3367
New longest path: 3382
New longest path: 3396
['2977', '1180', '1001', '1080', '1025', '107', '0', '101', '180', '194', '1', '119', '100', '163', '173', '166', '198', '1902', '1006', '1039', '1040', '1063', '1056', '1029', '1059', '1004', '1003', '1075', '1012', '1113', '1122', '1155', '1026', '1028', '1017', '1024', '1078', '1074', '1048', '1110', '1047', '1076', '1079', '1086', '1032', '1014', '1009', '1066', '1018', '1099', '1081', '1020', '1102', '1108', '1235', '1015', '1021', '1007', '1067', '1019', '1060', '1100', '1036', '1129', '1082', '2596', '1104', '1044', '1158', '1215', '1192', '1120', '1115', '1190', '1148', '1227', '1159', '1010', '1000', '1134', '1127', '1090', '1141', '1188', '1051', '1042', '1052', '1011', '1041', '1072', '1037', '1035', '1013', '1109', '1070', '1092', '1054', '1128', '1049', '1112', '1184', '1083', '1091', '1207', '1068', '1153', '1107', '1101', '1069', '1238', '1117', '1124', '112

(222.87624788284302,
 3396,
 ['2977',
  '1180',
  '1001',
  '1080',
  '1025',
  '107',
  '0',
  '101',
  '180',
  '194',
  '1',
  '119',
  '100',
  '163',
  '173',
  '166',
  '198',
  '1902',
  '1006',
  '1039',
  '1040',
  '1063',
  '1056',
  '1029',
  '1059',
  '1004',
  '1003',
  '1075',
  '1012',
  '1113',
  '1122',
  '1155',
  '1026',
  '1028',
  '1017',
  '1024',
  '1078',
  '1074',
  '1048',
  '1110',
  '1047',
  '1076',
  '1079',
  '1086',
  '1032',
  '1014',
  '1009',
  '1066',
  '1018',
  '1099',
  '1081',
  '1020',
  '1102',
  '1108',
  '1235',
  '1015',
  '1021',
  '1007',
  '1067',
  '1019',
  '1060',
  '1100',
  '1036',
  '1129',
  '1082',
  '1104',
  '1044',
  '1158',
  '1215',
  '1192',
  '1120',
  '1115',
  '1190',
  '1148',
  '1227',
  '1159',
  '1010',
  '1000',
  '1134',
  '1127',
  '1090',
  '1141',
  '1188',
  '1051',
  '1042',
  '1052',
  '1011',
  '1041',
  '1072',
  '1037',
  '1035',
  '1013',
  '1109',
  '1070',
  '1092',
  '1054',
  '1128',
  '1049',
  '1112'

(a) The length of the path returned by our algorithm is **3396**. 
<br>
(b) The path was **certified** by check_path.
<br>
(c) The code took **222.9** seconds to run.

In [440]:
G = graph()
# read in edges
G.Read_Edges("challenge1.txt")
G.Read_Vertex_Labels("challenge1.txt")

iterative_longest_path(G, 10)

Order 0 = 3191
Pi 0: 3191
New longest path: 3340
Order 0 = 2166
Pi 0: 2166
New longest path: 3363
Order 0 = 865
Pi 0: 865
Order 0 = 2340
Pi 0: 2340
Order 0 = 775
Pi 0: 775
Order 0 = 3560
Pi 0: 3560
Order 0 = 1910
Pi 0: 1910
Order 0 = 2624
Pi 0: 2624
Order 0 = 3544
Pi 0: 3544
Order 0 = 1440
Pi 0: 1440
['2166', '1190', '107', '0', '101', '180', '194', '1', '119', '100', '163', '173', '166', '198', '1902', '1006', '1039', '1040', '1063', '1056', '1029', '1059', '1004', '1003', '1075', '1012', '1113', '1025', '1080', '1001', '1095', '1300', '1179', '1114', '1155', '1026', '1028', '1017', '1024', '1078', '1074', '1048', '1110', '1047', '1076', '1079', '1086', '1032', '1014', '1009', '1066', '1018', '1099', '1081', '1020', '1102', '1108', '1235', '1015', '1021', '1007', '1067', '1019', '1060', '1100', '1036', '1129', '1082', '2596', '1104', '1044', '1158', '1215', '1192', '1120', '1115', '1221', '1035', '1010', '1000', '1134', '1127', '1090', '1122', '1237', '1030', '1043', '1045', '1183', '

(3.446492910385132,
 3363,
 ['2166',
  '1190',
  '107',
  '0',
  '101',
  '180',
  '194',
  '1',
  '119',
  '100',
  '163',
  '173',
  '166',
  '198',
  '1902',
  '1006',
  '1039',
  '1040',
  '1063',
  '1056',
  '1029',
  '1059',
  '1004',
  '1003',
  '1075',
  '1012',
  '1113',
  '1025',
  '1080',
  '1001',
  '1095',
  '1300',
  '1179',
  '1114',
  '1155',
  '1026',
  '1028',
  '1017',
  '1024',
  '1078',
  '1074',
  '1048',
  '1110',
  '1047',
  '1076',
  '1079',
  '1086',
  '1032',
  '1014',
  '1009',
  '1066',
  '1018',
  '1099',
  '1081',
  '1020',
  '1102',
  '1108',
  '1235',
  '1015',
  '1021',
  '1007',
  '1067',
  '1019',
  '1060',
  '1100',
  '1036',
  '1129',
  '1082',
  '1104',
  '1044',
  '1158',
  '1215',
  '1192',
  '1120',
  '1115',
  '1221',
  '1035',
  '1010',
  '1000',
  '1134',
  '1127',
  '1090',
  '1122',
  '1237',
  '1030',
  '1043',
  '1045',
  '1183',
  '1051',
  '1042',
  '1052',
  '1011',
  '1041',
  '1072',
  '1037',
  '1070',
  '1092',
  '1054',
  '1128',

### Challenge 2:

In [436]:
G = graph()
# read in edges
G.Read_Edges("challenge2.txt")
G.Read_Vertex_Labels("challenge2.txt")

iterative_longest_path(G, 500)

New longest path: 6876
New longest path: 6877
New longest path: 6879
New longest path: 6882
['2091', '1059', '1014', '1015', '102', '1007', '1036', '0', '1041', '1028', '1022', '1001', '1077', '1', '1004', '1020', '103', '1016', '1056', '1010', '1030', '1002', '1005', '1017', '100', '104', '1037', '10', '1043', '1057', '1000', '1011', '1092', '1003', '1084', '1027', '1019', '1013', '1038', '1042', '1012', '1068', '1040', '1032', '1025', '1127', '1105', '1026', '1021', '1044', '106', '1006', '1053', '1052', '1045', '1061', '1033', '1081', '1046', '1008', '1009', '1125', '1018', '1055', '1050', '1051', '1034', '1095', '107', '1035', '1047', '1023', '105', '1063', '1070', '1088', '1062', '1076', '1138', '1049', '1087', '1031', '1097', '1080', '1098', '1066', '1067', '1060', '1156', '1114', '112', '1069', '113', '1073', '1109', '1091', '1072', '1093', '1121', '1075', '1078', '1029', '1058', '1164', '1099', '1117', '1100', '1103', '1064', '1115', '1170', '1048', '1110', '1083', '109', '1146

(4392.827600955963,
 6882,
 ['2091',
  '1059',
  '1014',
  '1015',
  '102',
  '1007',
  '1036',
  '0',
  '1041',
  '1028',
  '1022',
  '1001',
  '1077',
  '1',
  '1004',
  '1020',
  '103',
  '1016',
  '1056',
  '1010',
  '1030',
  '1002',
  '1005',
  '1017',
  '100',
  '104',
  '1037',
  '10',
  '1043',
  '1057',
  '1000',
  '1011',
  '1092',
  '1003',
  '1084',
  '1027',
  '1019',
  '1013',
  '1038',
  '1042',
  '1012',
  '1068',
  '1040',
  '1032',
  '1025',
  '1127',
  '1105',
  '1026',
  '1021',
  '1044',
  '106',
  '1006',
  '1053',
  '1052',
  '1045',
  '1061',
  '1033',
  '1081',
  '1046',
  '1008',
  '1009',
  '1125',
  '1018',
  '1055',
  '1050',
  '1051',
  '1034',
  '1095',
  '107',
  '1035',
  '1047',
  '1023',
  '105',
  '1063',
  '1070',
  '1088',
  '1062',
  '1076',
  '1138',
  '1049',
  '1087',
  '1031',
  '1097',
  '1080',
  '1098',
  '1066',
  '1067',
  '1060',
  '1156',
  '1114',
  '112',
  '1069',
  '113',
  '1073',
  '1109',
  '1091',
  '1072',
  '1093',
  '1121',


(a) The length of the path returned by our algorithm is **6882**. 
<br>
(b) The path was **certified** by check_path.
<br>
(c) The code took **4392.8** seconds to run.

## Citations:
None